## Шаг 1: Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


## Шаг 2: Загрузка и первичный осмотр данных

In [ ]:
# Загрузка данных
data_path = '/content/covid-19-all.csv'
data = pd.read_csv(data_path)
print("Первые пять строк данных:")
print(data.head())

# Информация о данных для понимания структуры и типов данных
print("Информация о данных:")
data.info()


Первые пять строк данных:
   Country/Region Province/State  Latitude  Longitude  Confirmed  Recovered  \
0  Mainland China          Anhui   31.8257   117.2264        1.0        NaN   
1  Mainland China        Beijing   40.1824   116.4142       14.0        NaN   
2  Mainland China      Chongqing   30.0572   107.8740        6.0        NaN   
3  Mainland China         Fujian   26.0789   117.9874        1.0        NaN   
4  Mainland China          Gansu   36.0611   103.8343        NaN        NaN   

   Deaths        Date  
0     NaN  2020-01-22  
1     NaN  2020-01-22  
2     NaN  2020-01-22  
3     NaN  2020-01-22  
4     NaN  2020-01-22  
Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country/Region  6162 non-null   object 
 1   Province/State  3697 non-null   object 
 2   Latitude        6161 non-null   float64
 3 

##Шаг 3: Предобработка данных

In [ ]:
# Заполнение пропущенных значений
data.fillna(method='ffill', inplace=True)  # Заполняем пропущенные значения последним доступным значением

# Кодирование категориальных переменных
label_encoders = {}
categorical_columns = ['Country/Region', 'Province/State']  # Перечисляем категориальные колонки
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

# Выводим преобразованные данные
print("Данные после предобработки:")
print(data.head())


Данные после предобработки:
   Country/Region  Province/State  Latitude  Longitude  Confirmed  Recovered  \
0              94               6   31.8257   117.2264        1.0        NaN   
1              94              12   40.1824   116.4142       14.0        NaN   
2              94              31   30.0572   107.8740        6.0        NaN   
3              94              69   26.0789   117.9874        1.0        NaN   
4              94              71   36.0611   103.8343        1.0        NaN   

   Deaths        Date  
0     NaN  2020-01-22  
1     NaN  2020-01-22  
2     NaN  2020-01-22  
3     NaN  2020-01-22  
4     NaN  2020-01-22  


##Шаг 4: Масштабирование числовых признаков

In [ ]:
# Выбор числовых признаков для масштабирования
numeric_features = ['Confirmed', 'Deaths', 'Recovered']

# Масштабирование числовых признаков
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

# Вывод обработанных данных
print("Данные после масштабирования числовых признаков:")
print(data.head())


Данные после масштабирования числовых признаков:
   Country/Region  Province/State  Latitude  Longitude  Confirmed  Recovered  \
0              94               6   31.8257   117.2264  -0.120095        NaN   
1              94              12   40.1824   116.4142  -0.117447        NaN   
2              94              31   30.0572   107.8740  -0.119076        NaN   
3              94              69   26.0789   117.9874  -0.120095        NaN   
4              94              71   36.0611   103.8343  -0.120095        NaN   

   Deaths        Date  
0     NaN  2020-01-22  
1     NaN  2020-01-22  
2     NaN  2020-01-22  
3     NaN  2020-01-22  
4     NaN  2020-01-22  


##Шаг 5: Обучение модели KNN и подбор гиперпараметров

In [ ]:
from sklearn.impute import SimpleImputer

# Инициализация импьютера для заполнения NaN
imputer = SimpleImputer(strategy='mean')

# Применение импьютера к тренировочным данным
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Подбор гиперпараметров с помощью GridSearchCV
param_grid = { #словарь, содержащий параметры, которые нужно оптимизировать в KNN. Включает количество соседей (n_neighbors), тип весов (weights), и метрику расстояния (metric).
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy') #инициализация поиска по сетке с кросс-валидацией для оптимального выбора параметров
grid_search.fit(X_train_imputed, y_train) #запуск процесса обучения и поиска наилучших параметров модели на импутированных данных.

# Вывод лучших параметров и лучшего скора
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший скор по кросс-валидации:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Лучшие параметры: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Лучший скор по кросс-валидации: 0.9713937918441875


##Шаг 6: Обучение взвешенного KNN

In [ ]:
# Обучение взвешенного KNN с лучшими параметрами
weighted_knn = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'], #создание экземпляра классификатора KNN с лучшими параметрами, найденными в шаге 5.
                                     weights='distance',
                                     metric=grid_search.best_params_['metric'])
weighted_knn.fit(X_train_imputed, y_train) #обучение модели на импутированных обучающих данных.

# Оценка модели на тестовой выборке
y_pred = weighted_knn.predict(X_test_imputed) #прогнозирование на тестовых данных для оценки эффективности модели.

## Шаг 7: Сравнение моделей по метрике F1

In [ ]:
from sklearn.metrics import f1_score, classification_report

# Вычисление F1-метрики для базовой и взвешенной моделей
f1_weighted = f1_score(y_test, y_pred, average='weighted') #обеспечивает учет классов с различным количеством примеров
print("F1-метрика для взвешенной модели KNN:", f1_weighted)

# Дополнительный вывод отчета классификации для детального анализа
print(classification_report(y_test, y_pred))


F1-метрика для взвешенной модели KNN: 0.9779981648278621
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         3
           8       1.00      0.98      0.99        55
           9       1.00      1.00      1.00         5
          10       0.83      1.00      0.91         5
          11       1.00      1.00      1.00         7
          12       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         4
          14       1.00      1.00      1.00        10
          16       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         3
          19       1.00 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## Шаг 8: Получение предсказания от модели для нового объекта

In [ ]:
# Масштабирование нового объекта
new_object = np.array([[10000, 5000, 200]])  # Пример данных: "Confirmed", "Recovered", "Deaths"
new_object_scaled = scaler.transform(new_object)  # использование ранее созданного объекта scaler

# Предсказание модели на масштабированных данных
predicted_country = weighted_knn.predict(new_object_scaled)

# Декодирование предсказанной метки в название страны
predicted_country_name = label_encoders['Country/Region'].inverse_transform(predicted_country)

print("Предсказанная страна для нового объекта:", predicted_country_name[0])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 3 features, but KNeighborsClassifier is expecting 5 features as input.